In [ ]:
!pip install transformers sentencepiece

In [17]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)

In [ ]:
model = MBartForConditionalGeneration.from_pretrained(model_name)
model.to("cuda")

In [19]:
article_text = '«По итогам 2011 года чистый отток может составить примерно $80 млрд, в следующем году — около $20 млрд.\
При этом мы ожидаем, что со второго полугодия 2012 года начнется приток капитала», — заявил «Интерфаксу»\
замминистра экономического развития Андрей Клепач. Официальные прогнозы по выводу капитала из России становятся\
все пессимистичными: еще летом власти полагали, что из страны уйдет не более $35 млрд, в сентябре Минэкономразвития\
назвал цифру $50 млрд, в начале ноября Центробанк пересмотрел оценку до $70 млрд. Очередное изменение прогноза было ожидаемо:\
по расчетам Центробанка , за январь — октябрь чистый отток капитала достиг $64 млрд, причем в последние месяцы он ускорился:\
в сентябре он составил $14 млрд, в октябре — $13 млрд против среднего ежемесячного оттока в $6—8 млрд в первом полугодии.\
«После октябрьских данных Минэкономразвития вынуждено было изменить оценку, настаивать на $70 млрд означало ожидать\
серьезного замедления оттока капитала на непонятно каких причинах», — говорит главный экономист BNP Paribas Юлия Цепляева.\
«В последние два месяца отток капитала ускорится, на декабрь приходится значительная часть выплат по внешним долгам,\
что приводит к усилению оттока, особенно если они не рефинансируются новыми кредитами», — соглашается главный экономист\
ФК «Открытие» Владимир Тихомиров. Прогнозируемый Минэкономразвития отток капитала — один из самых высоких за последние 20 лет.\
Больше ушло лишь в 2008 году на фоне разрастания финансового кризиса и российско-грузинской войны — $133,7 млрд.\
В кризисный 2009 год из России утекло $56,1 млрд. Главный фактор ускорения оттока капитала в 2011 году — нестабильность\
на внешних финансовых рынках и рост опасений относительно второй волны рецессии. «Это реакция на неуверенность, которую\
генерирует Европа с долговыми проблемами. В случае новой волны глобальной турбулентности Россия — одна из самых уязвимых\
стран», — говорит Цепляева. Еще одна причина — ослабление рубля. «Привлекательность вложений снижается на фоне того,\
что рубль перестал укрепляться, а ставки по депозитам достаточно низкие. В результате экспортеры не полностью возвращают\
экспортную выручку», — говорит Тихомиров. Внутри страны эксперты не видят особых причин для бегства капитала.\
«Ситуация выглядит достаточно позитивно, очень хорошие макроэкономические результаты за год, особенно на фоне других стран.\
С политической точки зрения все достаточно понятно и предсказуемо, итог выборов очевиден», — говорит экономист\
ИК «Тройка Диалог» Антон Струченевский. Тем не менее политический фактор играет роль. «Бизнесу важно не только,\
кто будет президентом, он ждет ясности с перестановками в правительстве. В наших условиях административный ресурс\
важнее всего для успешности бизнеса», — говорит Цепляева, добавляя, что отток капитала продолжится до завершения\
президентских выборов.'

In [ ]:
input_ids = tokenizer(
    [article_text],
    max_length=600,
    truncation=True,
    return_tensors="pt",
)["input_ids"].to("cuda")
print(input_ids)

In [21]:
output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
)[0]
summary = tokenizer.decode(output_ids, skip_special_tokens=True)
print(summary)

В 2011 году из России утекло $80 млрд, прогнозирует Минэкономразвития. Это один из самых высоких за последние 20 лет. Причина — нестабильность на внешних финансовых рынках и рост опасений относительно второй волны рецессии. Внутри страны эксперты не видят особых причин для бегства капитала.


In [22]:
reference = 'В 2011 году из России уйдет $80 млрд, считают в Минэкономразвития. Менее месяца назад Центробанк давал оценку $70 млрд, повысив первоначальный прогноз вдвое. Отток капитала из страны усиливается из-за кризиса в Европе, а в декабре российским компаниям выплачивать внешние долги. На движение капитала повлияли и выборы: несмотря на их предсказуемость, бизнес хочет ясности с перестановками в правительстве.'

In [23]:
!pip install rouge

In [24]:
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references, '\n')
    print("Hyp:", predictions, '\n')

    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [25]:
calc_scores(reference, summary)

Count: 291
Ref: В 2011 году из России уйдет $80 млрд, считают в Минэкономразвития. Менее месяца назад Центробанк давал оценку $70 млрд, повысив первоначальный прогноз вдвое. Отток капитала из страны усиливается из-за кризиса в Европе, а в декабре российским компаниям выплачивать внешние долги. На движение капитала повлияли и выборы: несмотря на их предсказуемость, бизнес хочет ясности с перестановками в правительстве. 

Hyp: В 2011 году из России утекло $80 млрд, прогнозирует Минэкономразвития. Это один из самых высоких за последние 20 лет. Причина — нестабильность на внешних финансовых рынках и рост опасений относительно второй волны рецессии. Внутри страны эксперты не видят особых причин для бегства капитала. 

ROUGE:  {'rouge-1': {'r': 0.23529411764705882, 'p': 0.2857142857142857, 'f': 0.2580645111758586}, 'rouge-2': {'r': 0.08928571428571429, 'p': 0.11904761904761904, 'f': 0.10204081142857167}, 'rouge-l': {'r': 0.21568627450980393, 'p': 0.2619047619047619, 'f': 0.23655913483177257}